In [2]:
import k_general_kit.search_sort.search_sort as kgs
from k_general_kit.gen_obj import real

def part_list(L:list[real|str], start:int = 0, end:int = -1, ord:int = 0, pivot:int = -1)-> int: 
	"""It arranges a list L = L[start: end[ such that al elts < L[pivot] are before L[pivot] and all elts >= L[pivots] are after it, if ord = 0. It does the contrary else.
	It returns the final position of L[pivot]
	Rk: It optimises the space complexity by staying in the same list; however, time complexity worsens with the permutations."""
	assert start >= 0
	l = len(L)
	if end == -1:
		end = l
	assert end > start
	indices = {i:i for i in range(start, end)}
	
	if pivot != -1:
		assert start <= pivot <= end-1
		L[pivot], L[end-1] = L[end-1], L[pivot]
		indices[pivot], indices[end-1] = indices[end-1], indices[pivot]
	
	Pivot = L[end-1]
	pos = start #We initialise the position of the pivot = L[pivot] and we'll update it in the next part
	for i in range(start, end-1):
		if (L[i] < Pivot)+ord == 1:
			L[pos], L[i] = L[i], L[pos]
			indices[pos], indices[i] = indices[i], indices[pos]
			pos += 1
	L[pos], L[end-1] = L[end-1], L[pos]
	indices[pos], indices[end-1] = indices[end-1], indices[pos]
	return pos, indices

def k_min(tab:list[real|str], k:int, start:int = 0, end:int = -1, ord:int = 0, indices = {})-> int: 
	# print(indices)
	if end == -1:
		end = len(tab)
		if start == 0:
			indices = {i:i for i in range(start, end)}
	# print(start, end, k)
	assert end >= start + 1 and k <= end - start
	
	print(indices, tab[start:end], start, end)
	if end == start+1:
		return indices[start]
	# print(indices, tab[start:end])
	pivot_pos, indices_new = part_list(tab, start, end, ord)
	print(indices, indices_new, tab[start:end])
	indices = {i:indices[indices_new[i]] for i in range(start, end)}
	# print(indices, indices_new, tab[start:end])
	k_comp = pivot_pos - start + 1
	print("*", k, start, k_comp, pivot_pos)
	diff = k - k_comp
	if diff == 0:
		return indices[pivot_pos]
	elif diff < 0:
		# print(tab)
		# print("*", k, start, k_comp, pivot_pos)
		return k_min(tab, k, start, pivot_pos, ord, {i:indices[i] for i in range(start, pivot_pos)})
	return k_min(tab, diff, pivot_pos+1, end, ord, {i:indices[i] for i in range(pivot_pos+1, end)})

tab = [-1, 9, 48, 5, 10, 0]

print(tab[k_min(tab, 2)])

{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5} [-1, 9, 48, 5, 10, 0] 0 6
{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5} {0: 0, 1: 5, 2: 2, 3: 3, 4: 4, 5: 1} [-1, 0, 48, 5, 10, 9]
* 2 0 2 1
9


In [3]:
import math
import time

class Heap:
	def __init__(self, tab:list[float]):
		self.tab = tab
		self.size = len(tab)
		self.len = len(tab)
		
	def build_heap(self):
		self.size = self.len
		for i in range(pow(2, self.height())-2, -1, -1):
			self.heap_up(i)
		
	def heap_up(self, i):
		if i >= self.size:
			return
		
		l = self.left(i)
		r = self.right(i)
		max = i
		size = self.size
  
		if l < size and self.tab[l] > self.tab[max]:
			max = l
		if r < size and self.tab[r] > self.tab[max]:
			max = r
		
		if max != i:
			self.tab[i], self.tab[max] = self.tab[max], self.tab[i]
			self.heap_up(max)  
		
	def height_i(self, i):
		if i >= self.size:
			return 0
		return 1 + max(self.height_i(self.left(i)), self.height_i(self.right(i))) + 1    
	
	def height(self):
		return int(math.log2(self.len))
		
	def left(self, i:int):
		return 2*i+1
	
	def parent(self, i:int):
		return math.ceil(i/2) - 1
	
	def right(self, i:int):
		return 2*i+2
	
	def sort(self):
		self.build_heap()
		
		for i in range(self.len-1, 0, -1):
			self.tab[0], self.tab[i] = self.tab[i], self.tab[0]
			self.size -= 1
			self.heap_up(0)
	
	def print_smooth(self, space=1, arrows="↙↘", delay = 0.2): 
		width_max = max([len(str(x)) for x in self.tab])
	
		if space % 2 == 0:
			width = width_max if width_max % 2 == 0 else width_max + 1 
		else:
			width = width_max + 1 if width_max % 2 == 0 else width_max 
		
		height = self.height()
		
		fact = width + space
		seq = 1
		edge_spaces = [0]
		middle_spaces = [space]
		
		for i in range(height-1, 0, -1):
			seq *= 2
			edge_spaces.append(int(fact*(seq-1)/2))
			middle_spaces.append(int(fact*seq - width))
		
		seq *= 2
		edge_spaces.append(int(fact*(seq-1)/2))
		middle_spaces.append(0)
		edge_spaces.reverse()
		middle_spaces.reverse()
		tab = self.tab
		
		print(" " * edge_spaces[0] + str(tab[0]).center(width))
		seq1, seq2 = 1, 2
		for i in range(1, height+1):
			seq1 = seq2
			seq2 *= 2
			first_index = seq1-1
			last_index = min(seq2-1, self.len)
			edge_space = " " * edge_spaces[i]
			edge_space2 = int((width*3 + middle_spaces[i])/4) * " "
			middle_space = (int((width + middle_spaces[i])/2)-1) * " "
			pattern =  edge_space2 + arrows[0] + middle_space + arrows[1] + edge_space2
   
			print(edge_space + (" " * (middle_spaces[i])).join([pattern for j in range(min(self.len, math.floor(seq1/2)))]))
			time.sleep(delay)
			print(edge_space + (" " * middle_spaces[i]).join([str(tab[j]).center(width) for j in range(first_index, last_index)]))
			time.sleep(delay)		
 
	def print_tab(self):
		return self.tab.__str__()
	
	def __str__(self, space=1, arrows="↙↘"): 
		width_max = max([len(str(x)) for x in self.tab])
	
		if space % 2 == 0:
			width = width_max if width_max % 2 == 0 else width_max + 1 
		else:
			width = width_max + 1 if width_max % 2 == 0 else width_max 
		
		height = self.height()
		
		fact = width + space
		seq = 1
		edge_spaces = [0]
		middle_spaces = [space]
		
		for i in range(height-1, 0, -1):
			seq *= 2
			edge_spaces.append(int(fact*(seq-1)/2))
			middle_spaces.append(int(fact*seq - width))
		
		seq *= 2
		edge_spaces.append(int(fact*(seq-1)/2))
		middle_spaces.append(0)
		edge_spaces.reverse()
		middle_spaces.reverse()
		tab = self.tab
		
		string = " " * edge_spaces[0] + str(tab[0]).center(width) + "\n"
		seq1, seq2 = 1, 2
		for i in range(1, height+1):
			seq1 = seq2
			seq2 *= 2
			first_index = seq1-1
			last_index = min(seq2-1, self.len)
			edge_space = " " * edge_spaces[i]
			edge_space2 = int((width*3 + middle_spaces[i])/4) * " "
			middle_space = (int((width + middle_spaces[i])/2)-1) * " "
			pattern =  edge_space2 + arrows[0] + middle_space + arrows[1] + edge_space2
   
			string += edge_space + (" " * (middle_spaces[i])).join([pattern for j in range(min(self.len, math.floor(seq1/2)))]) + "\n"
			string += edge_space + (" " * middle_spaces[i]).join([str(tab[j]).center(width) for j in range(first_index, last_index)]) + "\n"
			
		return string
		
		
tab = [4, 7, 1, 0, 3, 5, -1, -3, 8, 7, 8, 9, -17]
heap = Heap(tab)

heap.print_smooth()

heap.sort()

heap.print_smooth()




	

               4 
           ↙       ↘     
       7               1 
     ↙   ↘           ↙   ↘   
   0       3       5       -1
  ↙ ↘     ↙ ↘     ↙ ↘     ↙ ↘  
 -3  8   7   8   9  -17
              -17
           ↙       ↘     
       -3              -1
     ↙   ↘           ↙   ↘   
   0       1       3       4 
  ↙ ↘     ↙ ↘     ↙ ↘     ↙ ↘  
 5   7   7   8   8   9 
